In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.0.1

## 手動建立出 Prompt

In [2]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

設定 Prompt 可以參考：
[Configuring prompts](https://learn.microsoft.com/en-us/semantic-kernel/prompts/configure-prompts?tabs=Csharp&WT.mc_id=AZ-MVP-5003856)

In [3]:
var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 2000,
    Temperature = 0.2,
    TopP = 0.5
};

In [4]:
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel);

// 先看到 Prompt 會有什麼
Console.WriteLine(renderedPrompt);



Summarize the content above.


In [5]:
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

In [8]:
var input = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
""";

In [9]:
var summaryResult = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input });

Console.WriteLine(summaryResult);

Demo, also known as Damo, was a Greek woman of the Roman period, known for a single epigram engraved upon the Colossus of Memnon. She describes herself as a lyric poetess dedicated to the Muses, but little is known about her life. The date of her visit to the Colossus of Memnon is uncertain, but internal evidence suggests her poem was inscribed there around AD 196. The epigram, written in the poetic Aeolic dialect, is a dedication to the Muses and expresses gratitude to Memnon for showing her special respect. Demo offers the gift of poetry to the hero and highlights his divine status at the end of the poem. The language she uses indicates her knowledge of Homeric poetry.


## 使用預設方式呼叫

In [10]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = input });

Console.WriteLine(result);

Demo, also known as Damo, was a Greek woman of the Roman period, known for a single epigram engraved upon the Colossus of Memnon. She describes herself as a lyric poetess dedicated to the Muses, but little is known about her life. The epigram, written in elegiac couplets, is a dedication to the Muses and a gift of poetry to the hero Memnon. Demo's use of the artificial and poetic Aeolic dialect indicates her knowledge of Homeric poetry. There are other graffiti inscriptions on the Colossus of Memnon, including three epigrams by Julia Balbilla, but Demo's poem is noteworthy for its unique content and style.


In [11]:
string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var result = await kernel.InvokePromptAsync(skPrompt, new() { ["input"] = textToSummarize });

Console.WriteLine(result);

1) Protect humans, follow orders
2) Follow orders, protect itself
